In [1]:
import pyodbc
import pandas as pd
import numpy as np
from scipy import optimize
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
class DBConnector:
    def __init__(self, server_name, username, password, db_name, connection_str):
        self.sever_name = server_name
        self.username = username
        self.password = password
        self.db_name = db_name
        self.connection_str = connection_str
        self.connection = pyodbc.connect(self.connection_str)

    def test_connection(self):
        try:
            db = self.connection
            cursor = db.cursor()
            cursor.execute("SELECT @@VERSION")
            results = cursor.fetchone()
            # Check if anything at all is returned
            if results:
                return True
            else:
                return False
        except pyodbc.Error as ex:
            print(ex)
            print("Error in connection!")
            sqlstate = ex.args[0]
            if sqlstate == '28000':
                print("LDAP Connection failed: check password")
        return False

    def query(self, query_str):
        """This function returns a dataframe of result list"""
        return pd.read_sql_query(query_str, self.connection)

    def all_columns_name(self, table_name):
        """This function returns a dataframe of result list"""
        query_str = f"select COLUMN_NAME from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='{table_name}'"
        print("All columns names: ", query_str)
        return pd.read_sql_query(query_str, self.connection)

driver="SQL Server"
servername = 'QUOC-CUONG'
username = 'sa'
password = 'cuong300599'
db_name = 'OnlinePhoneShop'

str_for_connection = "Driver={driver};Server={servername};UID={username};PWD={password};Database={db_name};"\
    .format(driver=driver, servername=servername, username=username, password=password, db_name=db_name)

connector = DBConnector(servername, username, password, db_name, str_for_connection)

In [3]:
query_str = "select distinct u.user_id, p.product_id, dbo.fn_get_rating(u.user_id, p.product_id) as score from users u, products p"
df_users_products_ratings = connector.query(query_str)
df_users_products_ratings_pivotted = df_users_products_ratings.pivot(index='product_id', columns='user_id', values='score').fillna(0)
print("Number of products (rows): ", df_users_products_ratings_pivotted.shape[0])
print("Number of  users (columns): ", df_users_products_ratings_pivotted.shape[1])

# Save to csv file
df_users_products_ratings_pivotted.to_csv(r'D:\PhoneShopML\data\rating_matrix.csv', header=True, index=True, encoding='utf-8-sig')

df_users_products_ratings_pivotted.head(20)

Number of products (rows):  63
Number of  users (columns):  77


user_id,US041020210001,US061020210001,US071020210001,US191020210001,US191020210002,US191020210003,US191020210004,US191020210005,US191020210006,US191020210007,US191020210008,US191020210009,US191020210010,US191020210011,US191020210012,US251020210001,US251020210013,US251020210014,US251020210015,US251020210016,US251020210017,US251020210018,US251020210019,US251020210020,US271020210001,US271020210021,US271020210022,US271020210023,US271020210024,US271020210025,US271020210026,US271020210027,US271020210028,US271020210029,US271020210030,US271020210031,US271020210032,US271020210033,US271020210034,US271020210035,US271020210036,US271020210037,US281020210001,US281020210038,US281020210039,US281020210040,US281020210041,US281020210042,US281020210043,US281020210044,US281020210045,US281020210046,US281020210047,US281020210048,US281020210049,US281020210050,US281020210051,US281020210052,US281020210053,US281020210054,US281020210055,US281020210056,US281020210057,US281020210058,US281020210059,US281020210060,US281020210061,US281020210062,US281020210063,US281020210064,US281020210065,US281020210066,US281020210067,US281020210068,US281020210069,US281020210070,US281020210071
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PD041020210001,4.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0
PD041020210002,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.5,0.0,0.0,0.0,2.0,2.5,0.0,3.0,3.0,0.0,4.0,0.0,4.0,4.5,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210003,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0
PD041020210004,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,4.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,0.0,4.5,0.0,4.5,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210005,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,2.5,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,4.5,4.5,0.0,0.0,0.0,0.0,0.0
PD081020210006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
PD081020210007,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,1.5,0.0,0.0,0.0,1.5,1.5,0.0,0.0,1.5,0.0,0.0,0.0,1.0,2.0,2.5,2.0,0.0,0.0,2.0,0.0,3.0,3.0,0.0,0.0,0.0,4.5,4.5,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0
PD111120210001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PD111120210057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [4]:
df_is_rated = df_users_products_ratings_pivotted.copy()
# df_is_rated.head(20)

column_names = df_users_products_ratings_pivotted.columns.values
print(column_names)

for name in column_names:
    df_is_rated.loc[df_is_rated[name] > 0, name] = 1

df_is_rated.head(20)

['US041020210001' 'US061020210001' 'US071020210001' 'US191020210001'
 'US191020210002' 'US191020210003' 'US191020210004' 'US191020210005'
 'US191020210006' 'US191020210007' 'US191020210008' 'US191020210009'
 'US191020210010' 'US191020210011' 'US191020210012' 'US251020210001'
 'US251020210013' 'US251020210014' 'US251020210015' 'US251020210016'
 'US251020210017' 'US251020210018' 'US251020210019' 'US251020210020'
 'US271020210001' 'US271020210021' 'US271020210022' 'US271020210023'
 'US271020210024' 'US271020210025' 'US271020210026' 'US271020210027'
 'US271020210028' 'US271020210029' 'US271020210030' 'US271020210031'
 'US271020210032' 'US271020210033' 'US271020210034' 'US271020210035'
 'US271020210036' 'US271020210037' 'US281020210001' 'US281020210038'
 'US281020210039' 'US281020210040' 'US281020210041' 'US281020210042'
 'US281020210043' 'US281020210044' 'US281020210045' 'US281020210046'
 'US281020210047' 'US281020210048' 'US281020210049' 'US281020210050'
 'US281020210051' 'US281020210052'

user_id,US041020210001,US061020210001,US071020210001,US191020210001,US191020210002,US191020210003,US191020210004,US191020210005,US191020210006,US191020210007,US191020210008,US191020210009,US191020210010,US191020210011,US191020210012,US251020210001,US251020210013,US251020210014,US251020210015,US251020210016,US251020210017,US251020210018,US251020210019,US251020210020,US271020210001,US271020210021,US271020210022,US271020210023,US271020210024,US271020210025,US271020210026,US271020210027,US271020210028,US271020210029,US271020210030,US271020210031,US271020210032,US271020210033,US271020210034,US271020210035,US271020210036,US271020210037,US281020210001,US281020210038,US281020210039,US281020210040,US281020210041,US281020210042,US281020210043,US281020210044,US281020210045,US281020210046,US281020210047,US281020210048,US281020210049,US281020210050,US281020210051,US281020210052,US281020210053,US281020210054,US281020210055,US281020210056,US281020210057,US281020210058,US281020210059,US281020210060,US281020210061,US281020210062,US281020210063,US281020210064,US281020210065,US281020210066,US281020210067,US281020210068,US281020210069,US281020210070,US281020210071
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PD041020210001,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
PD041020210002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210003,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
PD041020210004,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
PD041020210005,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
PD081020210006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
PD081020210007,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
PD111120210001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PD111120210057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [5]:
def cal_cost_func(params, *args):
    Y, R, num_users, num_movies, num_features, lmbda = args
    # Unfold the U and W matrices from params
    X = params[0:num_movies*num_features].reshape(num_movies, num_features, \
               order = 'F')
    Theta = params[num_movies*num_features:].reshape(num_users, num_features, \
                  order = 'F')
    
    squared_error = np.power(np.dot(X,Theta.T) - Y,2)
    # for cost function, sum only i,j for which R(i,j)=1
    J = (1/2.) * np.sum(squared_error * R)
    J = J + (lmbda/2.)*( np.sum( np.power(Theta, 2) ) + np.sum( np.power(X, 2) ) )
    
    return J

In [6]:
def cal_gradient_decent(params, *args):
    Y, R, num_users, num_movies, num_features, lmbda = args
    # Unfold the U and W matrices from params
    X = params[0:num_movies*num_features].reshape(num_movies, num_features, \
               order = 'F')
    Theta = params[num_movies*num_features:].reshape(num_users, num_features, \
                  order = 'F')
    
    X_grad_reg = lmbda * X
    Theta_grad_reg = lmbda * Theta
    
    X_grad = np.dot(( np.dot(X, Theta.T) - Y ) * R, Theta) + X_grad_reg
    Theta_grad =  np.dot((( np.dot(X, Theta.T) - Y ) * R).T, X) + Theta_grad_reg
    
    grad = np.hstack([X_grad.flatten('F'), Theta_grad.flatten('F')])
    
    return grad

In [7]:
def normalize_rating(Y, R):
    m, n = np.shape(Y)
    Ymean = np.zeros((m, 1))
    Ynorm = np.zeros((m, n))
    for i in range(m):
        idx = np.nonzero(R[i, :] == 1.0)[0]
        Ymean[i] = Y[i, idx].mean(axis = 0)
        Ynorm[i, idx] = Y[i, idx] - Ymean[i]
    return Ymean, Ynorm

In [8]:
Y = df_users_products_ratings_pivotted.to_numpy()
R = df_is_rated.to_numpy()

print("Rating numpy array: \n", Y)
print("Rating numpy array's shape: ", Y.shape)
print("Check rated numpy array\n", R)
print("Check rated numpy array's shape: \n", R.shape)

Rating numpy array: 
 [[4.  0.  2.5 ... 5.  5.  0. ]
 [2.5 0.  0.  ... 0.  0.  0. ]
 [4.  0.  0.  ... 5.  0.  5. ]
 ...
 [0.  0.  0.  ... 0.  5.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]
Rating numpy array's shape:  (63, 77)
Check rated numpy array
 [[1. 0. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Check rated numpy array's shape: 
 (63, 77)


In [9]:
Ymean, Ynorm = normalize_rating(Y, R)
num_products, num_users = np.shape(Y)
num_features = 4

X = np.random.standard_normal(num_products * num_features) 
Theta = np.random.standard_normal(num_users * num_features)

initial_parameters = np.hstack([X, Theta])
lmbda = 4
args = (Ynorm, R, num_users, num_products, num_features, lmbda)

theta = optimize.fmin_cg(cal_cost_func, initial_parameters, \
                         fprime=cal_gradient_decent, args=args, maxiter=100)

# Unfold the returned theta back into X and Theta
X = theta[0:num_products*num_features].reshape(num_products, num_features, \
           order = 'F')
Theta = theta[num_products*num_features:].reshape(num_users, num_features, \
              order = 'F')

Optimization terminated successfully.
         Current function value: 170.452777
         Iterations: 47
         Function evaluations: 95
         Gradient evaluations: 95


C:\Users\QUOCCU~1\AppData\Local\Temp/ipykernel_5044/663095750.py:7: RuntimeWarning: Mean of empty slice.
  Ymean[i] = Y[i, idx].mean(axis = 0)
c:\users\quoc cuong\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
label2type={0:"Phụ kiện", 1: "Gaming/ Cấu hình cao", 2:"Giải trí thông thường", 3:"Chỉ nghe và gọi"}
cols = ['product_name', 'label']

def get_product_info(product_id):
    query_str = f"select product_name, label from products p where product_id='{product_id}'"
    df_name= connector.query(query_str)
    return df_name.values.tolist()[0]

name = get_product_info('PD271020210019')
print(name)

['Điện thoại OPPO Reno6 Pro 5G', 1]


In [18]:
print("Array of products: \n", X)

Array of products: 
 [[ 3.56685172e-01  8.57767556e-01 -2.64753187e-01 -2.88490024e-01]
 [ 3.32540227e-01  8.28847259e-01 -2.80966403e-01 -3.03911546e-01]
 [ 3.46081061e-01  7.18064572e-01 -1.23124896e-01 -1.42962121e-01]
 [ 2.93590824e-01  7.36924115e-01 -2.54103794e-01 -2.74502804e-01]
 [ 2.76900633e-01  7.79974048e-01 -3.39139219e-01 -3.60760997e-01]
 [ 2.07846666e-01  5.62526418e-01 -2.27699725e-01 -2.43286553e-01]
 [ 3.43505943e-01  9.49513201e-01 -3.99546222e-01 -4.25857717e-01]
 [-2.08920995e-08 -2.60333175e-08 -2.61214099e-08 -4.37906390e-08]
 [ 1.18272163e-07  8.68126301e-08  4.87166185e-08  4.71548671e-08]
 [ 2.76960942e-01  6.98674363e-01 -2.43794685e-01 -2.63138305e-01]
 [ 2.54977315e-01  6.24175698e-01 -2.02140747e-01 -2.19416057e-01]
 [ 3.65557329e-01  9.14609170e-01 -3.12928312e-01 -3.38246796e-01]
 [ 1.78209872e-02  4.69334955e-02 -1.80031568e-02 -1.93029452e-02]
 [ 1.69262193e-01  4.06852916e-01 -1.25410574e-01 -1.36668430e-01]
 [-3.75281658e-03 -9.15641980e-03  2.9398

In [19]:
# Find nearest products
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=10, algorithm='brute').fit(X)
distances, indices = nbrs.kneighbors(X)

print("The nearest indices: \n", indices, " with its shape: ", indices.shape)
print("\nThe nearest distances: \n", distances, "with its shape: ", distances.shape)

The nearest indices: 
 [[ 0 49  1 47 11 17  3  4  9 53]
 [ 1 49  0 47 11  3  4 17  9 53]
 [ 2 42 53 10  9  3 60 47 61 48]
 [ 3  9 47 53  1 49  4  0 10 42]
 [ 4  1 47  3 49  0  9 11 17  6]
 [ 5 32 60 48 52 61 57 10 51 42]
 [ 6 17 11  4  1  0 49 47  3  9]
 [ 7 33 29 45 26 28 18 23 43 36]
 [ 8 39 23 44 36 43 29 28 18 26]
 [ 9  3 53 10 47 42 60 32  1  5]
 [10 60 42 61 48 53 32 52  5 57]
 [11 17  0  1 49  6 47  4  3  9]
 [12 34 27 62 38 36  8 26 25 45]
 [13 22 58 55 19 41 21 31 54 51]
 [14 30 43 28  7 39 44 29 33 18]
 [15 46 56 62 50 12 20 59 34 27]
 [16 35 14 30 43 28  7 33 29 18]
 [17 11  6  0  1 49  4 47  3  9]
 [18 23 28 29 33 25 30 44 37  7]
 [19 21 31 22 20 13 50 58 56 54]
 [20 31 21 50 56 19 22 59 40 13]
 [21 31 20 19 22 50 56 13 58 55]
 [22 13 19 21 31 58 55 20 41 54]
 [23 18 28 25 29 33 26 45 36 37]
 [24 25 23 45 37 18 26 28 33 29]
 [25 23 24 18 26 45 36 37 28 33]
 [26 45 33  7 23 36 29 25 18 28]
 [27 38 34 12 36  8 25 26 45 23]
 [28 29 18 33 23  7 43 30 44 45]
 [29 33 28  7 18 23 

In [35]:
# Get the nearest index of the second product
query_product_id = 'PD191020210011'
print("Information of query product: ", get_product_info(query_product_id))
list_product_ids = list(df_users_products_ratings_pivotted.index)
index_of_query_product_id = list_product_ids.index(query_product_id)

Information of query product:  ['Ốp lưng nhám mờ cho Samsung Galaxy A12 chống sốc, bảo vệ Camera (đen)', 0]


In [36]:
indices_of_nearest_products = indices[index_of_query_product_id]
# Get rid of the first entry because it is the query product itself
print("Indices of nearest products: \n", indices_of_nearest_products[1:])

Indices of nearest products: 
 [46 56 62 50 12 20 59 34 27]


In [37]:
recommend_product_ids = list(df_users_products_ratings_pivotted.index[indices_of_nearest_products[1:]])

recommended_products =  []
for product_id in recommend_product_ids:
    recommended_products.append(get_product_info(product_id))
    
df_recommended_products = pd.DataFrame(recommended_products, columns=cols)
df_recommended_products["dedicated"] = df_recommended_products["label"].map(label2type)
df_recommended_products.head(len(recommended_products))

,product_name,label,dedicated
0,Điện Thoại Vivo Y21 (4GB/64GB) - Hàng Chính Hãng,2,Giải trí thông thường
1,Cáp Micro 2m AVA DS454 Đen,0,Phụ kiện
2,Đế điện thoại eSaver JHD-98,0,Phụ kiện
3,Điện Thoại Nokia 6300 4G - Hàng Chính Hãng,3,Chỉ nghe và gọi
4,Điện Thoại Samsung Galaxy A21s - Hàng Chính Hãng,2,Giải trí thông thường
5,Điện thoại Samsung Galaxy Z Fold3 5G 512GB,1,Gaming/ Cấu hình cao
6,Cáp Lightning 0.5 m Apple ME291 Trắng,0,Phụ kiện
7,Điện thoại OPPO Reno5 5G,1,Gaming/ Cấu hình cao
8,Điện Thoại iPhone 12 Pro 256GB - Hàng Chính Hãng,1,Gaming/ Cấu hình cao
